In [2]:
import pandas as pd
import logging
import ast
import numpy as np
from sentence_transformers import SentenceTransformer

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 100)

In [4]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [5]:
def preprocess_product_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess product dataset
    """
    df.fillna('', inplace=True)
    df['product_gender'] = df['product_gender'].apply(ast.literal_eval).apply(lambda x: ','.join(x)) 
    df['product_style'] = df['product_style'].apply(ast.literal_eval).apply(lambda x: ', '.join(x)) 
    df['product_note'] = df['product_note'].apply(ast.literal_eval).apply(lambda x: ', '.join(x)) 
    df['combined_text'] = (
        df['full_name'].str.strip() + ' ' +
        df['product_gender'].str.strip() + ' ' +
        df['brand'].str.strip() + ' ' +
        df['description'].str.strip() + ' ' +
        df['origin'].str.strip() + ' ' +
        df['product_style'].str.strip() + ' ' +
        df['product_note'].str.strip()
    )

    numeric_cols = df.drop(['parent_id'], axis=1).dtypes[(df.dtypes=='int64') | (df.dtypes=='float64')].index.values
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df = df[df['price'] >= 0]
    df = df[df['product_gender'] != '']
    df = df[df['description']!='']    
    return df

In [6]:
logger.info("Loading datasets...")
df = pd.read_csv('../data/namperfume_product.csv')
processed = preprocess_product_data(df)
print(processed.shape)
processed.head(1)

2025-03-13 03:34:41,986 - INFO - Loading datasets...


(1538, 25)


,id,name,full_name,parent_handle,sku,production_year,origin,price,compare_at_price,parent_id,total_quantity,sold_quantity,brand,product_gender,product_style,product_note,rate,count_rate,count_rate_1,count_rate_2,count_rate_3,count_rate_4,count_rate_5,description,combined_text
0,1003772321,Dsquared2 Icon Pour Homme,Dsquared2 Icon Pour Homme - 100ml Tester,dsquared2-icon-pour-homme,110100204700,2024,Ý,2200000,0,1003772317,0,0,DSQUARED2,Nam,"Tinh tế, Nam tính, Cuốn hút","Hương gừng, Cây xô thơm",0.0,0,0,0,0,0,0,"Hương đầu: Gừng, Cam chanh Hương giữa: Hoa tulip, Xô thơm, Phong lữ, Oải hương Hương cuối: Akiga...","Dsquared2 Icon Pour Homme - 100ml Tester Nam DSQUARED2 Hương đầu: Gừng, Cam chanh Hương giữa: Ho..."


In [7]:
def create_product_embeddings(df: pd.DataFrame, model_name: str = "all-MiniLM-L6-v2") -> np.ndarray:
    """
    Create embeddings for product descriptions
    """
    logger.info(f"Creating embeddings using {model_name}...")
    
    model = SentenceTransformer(model_name)
    texts = df.apply(
        lambda x: f"{x['combined_text']}", 
        axis=1
    ).tolist()
    embeddings = model.encode(texts,show_progress_bar=True,batch_size=32)
    
    return embeddings

In [8]:
import chromadb

chroma_client = chromadb.PersistentClient(path="chroma_db")

collection = chroma_client.get_or_create_collection(name="documents")

processed_list = processed.to_dict(orient="records")
embed = create_product_embeddings(processed).tolist()
for i, product in enumerate(processed_list):
    collection.add(
        ids=[str(product['id'])],  
        embeddings=[embed[i]],
        metadatas=[{
            "name": product["name"],
            "description": product["description"],
            "production_year": product["production_year"],
            'origin': product['origin'], 
            'price': product['price'], 
            'compare_at_price': product['compare_at_price'], 
            'sold_quantity': product['sold_quantity'],
            'brand': product['brand'],
            'product_gender': product['product_gender'], 
            'product_note': product['product_note'], 
            'rate': product['rate'], 
            'count_rate': product['count_rate'], 
            'count_rate_1': product['count_rate_1'], 
            'count_rate_2': product['count_rate_2'], 
            'count_rate_3': product['count_rate_3'],
            'count_rate_4': product['count_rate_4'],
            'count_rate_5': product['count_rate_5']
            
            }]
    )

print(f"Stored {collection.count()} documents in ChromaDB.")


2025-03-13 03:35:05,124 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-03-13 03:35:05,259 - INFO - Creating embeddings using all-MiniLM-L6-v2...
2025-03-13 03:35:05,301 - INFO - Use pytorch device_name: mps
2025-03-13 03:35:05,301 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
Batches: 100%|██████████| 49/49 [00:10<00:00,  4.77it/s]


Stored 1538 documents in ChromaDB.


In [19]:
import google.generativeai as genai
import os

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

In [14]:
def rag_query(question):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    query_embedding = model.encode([question], convert_to_tensor=True).tolist()

    results = collection.query(
        query_embeddings=query_embedding,
        n_results=3  
    )
    context = "\n\n".join(['\n'.join([res['name'],res["description"]]) for res in results["metadatas"][0]])
    
    prompt = f"Using the following context:\n{context}\nAnswer the question: {question}"
    response = genai.GenerativeModel("gemini-2.0-flash").generate_content(prompt)

    return response.text





In [20]:
# Example query
answer = rag_query("Nốt hương của Bleu Channel?")
print("Generated Response:", answer)

2025-03-13 04:00:31,841 - INFO - Use pytorch device_name: mps
2025-03-13 04:00:31,843 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
Batches: 100%|██████████| 1/1 [00:00<00:00, 78.59it/s]


Generated Response: Dựa trên đoạn văn bạn cung cấp, có vẻ như bạn đang hỏi về các nốt hương của cả hai phiên bản Bleu de Chanel: EDT (Eau de Toilette) và EDP (Eau de Parfum). Dưới đây là chi tiết:

**Bleu de Chanel (Eau de Toilette):**

*   **Hương đầu:** Quả bưởi, Quả chanh vàng, Bạc hà, Tiêu hồng.
*   **Hương giữa:** Gừng, Nhục đậu khấu, Hoa nhài, Iso E Super.
*   **Hương cuối:** Nhang, Cỏ hương bài, Gỗ tuyết tùng, Gỗ đàn hương, Hoắc hương, Nhựa hương Labdanum, Xạ hương trắng.

**Bleu de Chanel Eau de Parfum:**

*   **Hương đầu:** Quả bưởi, Quả chanh vàng, Bạc hà, Tiêu hồng, Cam Bergamot, Aldehydes, Hạt rau mùi.
*   **Hương giữa:** Gừng, Nhục đậu khấu, Hoa nhài, Quả dưa.
*   **Hương cuối:** Nhang, Nhựa hổ phách, Gỗ tuyết tùng, Gỗ đàn hương, Hoắc hương, Nhựa Labdanum, Nhựa Hổ phách.
